# 교량을 기준으로 가장 가까운 하천 관측소 매칭

In [1]:
import pandas as pd

In [2]:
gyo_df = pd.read_csv(r"C:\Users\Shin\Desktop\for-kr-pjt\NoonMap-Project\data\file_data\교량.csv")

In [3]:
gyo_df

,Unnamed: 0,교량명,시도명,시군구명,교량시작점위도,교량시작점경도,교량높이
0,0,진위교,경기도,평택시,37.100899,127.064133,4.6
1,1,풍계교,경기도,이천시,37.133443,127.602141,4.5
2,2,태평교,경기도,여주시,37.200325,127.545522,3.0
3,3,장평교,경기도,이천시,37.225055,127.524158,4.0
4,4,풍계교,경기도,이천시,37.149231,127.626386,7.0
...,...,...,...,...,...,...,...
5438,35183,인천신항교,인천광역시,연수구,37.348628,126.659157,17.3
5439,35184,아암1교,인천광역시,연수구,37.419259,126.634377,5.1
5440,35185,아암2교,인천광역시,연수구,37.429930,126.623233,5.0
5441,35186,아암3교,인천광역시,중구,37.431460,126.614248,5.3


In [4]:
gyo_df = gyo_df.drop(columns=['Unnamed: 0'])

In [5]:
gyo_df

,교량명,시도명,시군구명,교량시작점위도,교량시작점경도,교량높이
0,진위교,경기도,평택시,37.100899,127.064133,4.6
1,풍계교,경기도,이천시,37.133443,127.602141,4.5
2,태평교,경기도,여주시,37.200325,127.545522,3.0
3,장평교,경기도,이천시,37.225055,127.524158,4.0
4,풍계교,경기도,이천시,37.149231,127.626386,7.0
...,...,...,...,...,...,...
5438,인천신항교,인천광역시,연수구,37.348628,126.659157,17.3
5439,아암1교,인천광역시,연수구,37.419259,126.634377,5.1
5440,아암2교,인천광역시,연수구,37.429930,126.623233,5.0
5441,아암3교,인천광역시,중구,37.431460,126.614248,5.3


In [6]:
gyo_df = gyo_df.rename({'교량명':'bridge_name', '시도명':'address', '시군구명':'etc_address',
        '교량시작점위도':'latitude', '교량시작점경도':'longitude', '교량높이':'brid_height_origin'}, axis=1)

In [7]:
location_start = [[gyo_df["latitude"][i],gyo_df["longitude"][i]] for i in range(len(gyo_df))]

In [8]:
gyo_df["location_start"] = location_start

In [9]:
from haversine import haversine

In [10]:
wl_station_df = pd.read_csv(r"C:\Users\Shin\Desktop\for-kr-pjt\NoonMap-Project\data\file_data\수위_관측소.csv")

In [11]:
wl_station_df.isna().sum()

Unnamed: 0      0
station_code    0
station_name    0
address         0
etc_address     0
longitude       0
latitude        0
dtype: int64

In [12]:
wl_station_df = wl_station_df.drop(["Unnamed: 0"], axis=1)

In [13]:
def location(x,i: int,list_name):
    list0 = str(x['latitude'][i]).split('-')
    list1 = str(x['longitude'][i]).split('-')
    DD1 = int(list0[0]) + int(list0[1])/60 + int(list0[2])/3600
    DD2 = int(list1[0]) + int(list1[1])/60 + int(list1[2])/3600
    list_name.append([DD1,DD2])

In [14]:
loc_list_wl = []

In [15]:
for i in range(0, len(wl_station_df)):
    location(wl_station_df,i, loc_list_wl)

In [16]:
loc_list_wl

[[37.10666666666667, 127.43361111111112],
 [37.115, 127.63527777777779],
 [37.1625, 127.63416666666667],
 [37.202222222222225, 127.71916666666667],
 [37.23111111111111, 127.71388888888889],
 [37.25472222222222, 127.68083333333334],
 [37.183055555555555, 127.68666666666667],
 [37.27305555555555, 127.68333333333334],
 [37.28361111111111, 127.68472222222222],
 [37.29611111111111, 127.64750000000001],
 [37.32611111111112, 127.61555555555555],
 [37.33388888888889, 127.57305555555556],
 [37.328611111111115, 127.60333333333332],
 [37.26694444444444, 127.46083333333334],
 [37.46416666666667, 127.54305555555555],
 [37.33305555555556, 127.535],
 [37.4225, 127.62972222222221],
 [37.3825, 127.5625],
 [37.38111111111111, 127.54972222222221],
 [37.401111111111106, 127.5375],
 [37.401111111111106, 127.53777777777778],
 [37.40833333333333, 127.54138888888889],
 [37.38861111111111, 127.4888888888889],
 [37.46611111111111, 127.5275],
 [37.48638888888889, 127.49],
 [37.51166666666666, 127.65111111111112]

In [17]:
wl_station_df["location"] = loc_list_wl

In [18]:
wl_station_code = []
def closest_station1(x):
    for i in range(len(gyo_df)):
        min_index = 0
        for j in range(len(wl_station_df)):
            a = haversine(gyo_df['location_start'][i], wl_station_df['location'][j])
            if j == 0:
                min_dist = a
            else:
                if a<=min_dist:
                    min_dist= a
                    min_index = j
        wl_station_code.append(wl_station_df['station_code'][min_index])

In [19]:
closest_station1(wl_station_df)

In [20]:
gyo_df["wl_station_code"] = wl_station_code

In [21]:
gyo_df

,bridge_name,address,etc_address,latitude,longitude,brid_height_origin,location_start,wl_station_code
0,진위교,경기도,평택시,37.100899,127.064133,4.6,"[37.100899, 127.064133]",1101663
1,풍계교,경기도,이천시,37.133443,127.602141,4.5,"[37.133443, 127.602141]",1007605
2,태평교,경기도,여주시,37.200325,127.545522,3.0,"[37.200325, 127.545522]",1007615
3,장평교,경기도,이천시,37.225055,127.524158,4.0,"[37.225055, 127.524158]",1007645
4,풍계교,경기도,이천시,37.149231,127.626386,7.0,"[37.149231, 127.626386]",1007615
...,...,...,...,...,...,...,...,...
5438,인천신항교,인천광역시,연수구,37.348628,126.659157,17.3,"[37.34862800000001, 126.659157]",1202660
5439,아암1교,인천광역시,연수구,37.419259,126.634377,5.1,"[37.419259, 126.634377]",1201653
5440,아암2교,인천광역시,연수구,37.429930,126.623233,5.0,"[37.42993, 126.623233]",1201653
5441,아암3교,인천광역시,중구,37.431460,126.614248,5.3,"[37.43146, 126.614248]",1201653


In [22]:
gyo_df.to_csv("교량과_가까운_수위관측소_매칭.csv")